# Day 23

## Part One

<!-- Insert Markdown Here -->

---

In [23]:
# Initialise
from queue import PriorityQueue

with open('Day23.in') as f:
    spaces = set()
    amphipods = set()
    for y, row in enumerate(reversed(f.readlines())):
        for x, char in enumerate(row):
            if char not in ('#', ' ', '\n'):
                spaces.add((x,y))
                if char != '.':
                    # An amphipod is a (char, (x,y), moved) tuple where:
                    #    * char is the letter representation of the amphipod (A,B,C,D)
                    #    * (x,y) is its current position
                    #    * moved is a boolean indicating if it has been moved yet
                    amphipods.add((char, (x, y), False))
    energy = {
        'A':1,
        'B':10,
        'C':100,
        'D':1000
    }

In [24]:
def adjacent(pt, spaces=spaces):
    """pt is an x,y pair"""
    adj = []
    x, y = pt
    
    if (p:=(x-1, y)) in spaces:
        adj.append(p)
    if (p:=(x+1, y)) in spaces:
        adj.append(p)
    if (p:=(x, y-1)) in spaces:
        adj.append(p)
    if (p:=(x, y+1)) in spaces:
        adj.append(p)
    
    return set(adj)

    
def cost(char, from_, to_):
    """
    Cost needs to be a little weird because it's not the straight manhattan distance
    """
    xdist = abs(from_[0]-to_[0])
    if xdist == 0:
        ydist = abs(from_[1]-to_[1])
    else:
        ydist = abs(3 - from_[1]) + abs(3 - to_[1])
    
    return energy[char]*(xdist+ydist)

def adjacent(pt, spaces=spaces):
    """pt is an x,y pair"""
    adj = []
    x, y = pt
    
    if (p:=(x-1, y)) in spaces:
        adj.append(p)
    if (p:=(x+1, y)) in spaces:
        adj.append(p)
    if (p:=(x, y-1)) in spaces:
        adj.append(p)
    if (p:=(x, y+1)) in spaces:
        adj.append(p)
    
    return set(adj)


doorways = set([p for p in spaces if len(adjacent(p))==3])


def occupado(amphipods=amphipods):
    return set([pt for _, pt, _ in amphipods])


def destination(amphipod, amphipods=amphipods):
    letter, (x,y), _ = amphipod
    column = (ord(letter)-65)*2 + 3
    
    holes = set(range(1,3))
    
    for char, (col,h), _ in amphipods:
        if (char == letter) and (col == column) and (y != h):
            holes -= {h}
    
    return (column, min(holes))


def heuristic(amphipod, amphipods=amphipods):
    letter, (x,y), _ = amphipod
    column = (ord(letter)-65)*2 + 3
    
    holes = set(range(1,3))
    
    for char, (col,h), _ in amphipods:
        if (char == letter) and (col == column) and (y != h):
            holes -= {h}
    
    return (column, max(holes))


def connected(pt, occupied, spaces=spaces):
    pts = adjacent(pt) - occupied
    new = pts.copy()
    
    while new:
        temp = set()
        for p in new:
            temp |= adjacent(p) - occupied
        
        new = temp - pts
        pts |= temp
    
    return pts


def possible(amphipod, amphipods=amphipods, spaces=spaces, doorways=doorways):
    char, pt, moved = amphipod
    occupied = occupado(amphipods)
    home = destination(amphipod, amphipods)
    conn = connected(pt, occupied, spaces)
    poss = set()
    
    for x,y in conn:
        if y < 3:
            if (x,y) == home:
                poss.add((x,y))
            else:
                continue
        elif ((x,y) not in doorways):
            poss.add((x,y))
    
    if moved:
        return {home} if home in poss else set()
    else:
        return poss


def next_states(amphipod_state, spaces=spaces, doorways=doorways):
    new = set()
    t, n, c, *A = amphipod_state
    
    for a in A:
        letter, pt, moveable = a
        for p in possible(a, A, spaces, doorways):
            new_cost = c + cost(letter, pt, p)
            new.add(
                hash_state(
                    (set(A) - {a}) | {(letter, p, True)}, new_cost, n+1
                )
            )
    return new


def finito(amphipods):
    for a in amphipods:
        _, pt, _ = a
        if destination(a, amphipods) != pt:
            return False
    else:
        return True
    


def hash_state(amphipods, c, n):
    
    t = c
    
    for a in amphipods:
        char, pt, moved = a
        t += cost(char, pt, heuristic(a, amphipods))
    
    return tuple([t, n, c] + sorted(amphipods))

In [19]:
amphipods

{('A', (3, 1), False),
 ('A', (9, 1), False),
 ('B', (3, 2), False),
 ('B', (7, 2), False),
 ('C', (5, 2), False),
 ('C', (7, 1), False),
 ('D', (5, 1), False),
 ('D', (9, 2), False)}

In [21]:
states = PriorityQueue()
seen = set()
finished = False
best = 1e10
i = 0

first_state = hash_state(amphipods, 0, 0)
states.put(first_state)

while not finished:
    i += 1
    state = states.get_nowait()
    
    if state[2] > best:
        finished = True
    
    new = next_states(state)
    
    for n in new:
        if n[2]==n[0]:
            if n[2] < best:
                fin = n
                best = n[2]
                break
        elif n[3:] not in seen:
            seen.add(n[3:])
            states.put(n)
        else:
            pass
    
print(best)

12521


In [22]:
i

25080

In [10]:
states.qsize()

18874

In [9]:
fin

(12541,
 10,
 12541,
 ('A', (3, 1), False),
 ('A', (3, 2), True),
 ('B', (5, 1), True),
 ('B', (5, 2), True),
 ('C', (7, 1), False),
 ('C', (7, 2), True),
 ('D', (9, 1), True),
 ('D', (9, 2), True))

In [25]:
first_state = hash_state(amphipods, 0, 0)

In [26]:
next_states(first_state)

{(8590,
  1,
  20,
  ('A', (3, 1), False),
  ('A', (9, 1), False),
  ('B', (3, 2), False),
  ('B', (6, 3), True),
  ('C', (5, 2), False),
  ('C', (7, 1), False),
  ('D', (5, 1), False),
  ('D', (9, 2), False)),
 (8590,
  1,
  20,
  ('A', (3, 1), False),
  ('A', (9, 1), False),
  ('B', (4, 3), True),
  ('B', (7, 2), False),
  ('C', (5, 2), False),
  ('C', (7, 1), False),
  ('D', (5, 1), False),
  ('D', (9, 2), False)),
 (8590,
  1,
  200,
  ('A', (3, 1), False),
  ('A', (9, 1), False),
  ('B', (3, 2), False),
  ('B', (7, 2), False),
  ('C', (6, 3), True),
  ('C', (7, 1), False),
  ('D', (5, 1), False),
  ('D', (9, 2), False)),
 (8610,
  1,
  20,
  ('A', (3, 1), False),
  ('A', (9, 1), False),
  ('B', (2, 3), True),
  ('B', (7, 2), False),
  ('C', (5, 2), False),
  ('C', (7, 1), False),
  ('D', (5, 1), False),
  ('D', (9, 2), False)),
 (8610,
  1,
  20,
  ('A', (3, 1), False),
  ('A', (9, 1), False),
  ('B', (3, 2), False),
  ('B', (8, 3), True),
  ('C', (5, 2), False),
  ('C', (7, 1), F

---

## Part Two

<!-- Insert Markdown Here -->

---

In [41]:
new_map = """#############
#...........#
###C#B#A#D###
  #D#C#B#A#
  #D#B#A#C#
  #B#C#D#A#
  #########"""


spaces = set()
amphipods = set()
for y, row in enumerate(reversed(new_map.split('\n'))):
    for x, char in enumerate(row):
        if char not in ('#', ' ', '\n'):
            spaces.add((x,y))
            if char != '.':
                # An amphipod is a (char, (x,y), moved) tuple where:
                #    * char is the letter representation of the amphipod (A,B,C,D)
                #    * (x,y) is its current position
                #    * moved is a boolean indicating if it has been moved yet
                amphipods.add((char, (x, y), False))

In [42]:
def cost(char, from_, to_):
    """
    Cost needs to be a little weird because it's not the straight manhattan distance
    """
    xdist = abs(from_[0]-to_[0])
    if xdist == 0:
        ydist = abs(from_[1]-to_[1])
    else:
        ydist = abs(5 - from_[1]) + abs(5 - to_[1])
    
    return energy[char]*(xdist+ydist)

def adjacent(pt, spaces=spaces):
    """pt is an x,y pair"""
    adj = []
    x, y = pt
    
    if (p:=(x-1, y)) in spaces:
        adj.append(p)
    if (p:=(x+1, y)) in spaces:
        adj.append(p)
    if (p:=(x, y-1)) in spaces:
        adj.append(p)
    if (p:=(x, y+1)) in spaces:
        adj.append(p)
    
    return set(adj)


doorways = set([p for p in spaces if len(adjacent(p))==3])


def occupado(amphipods=amphipods):
    return set([pt for _, pt, _ in amphipods])

def destination(amphipod, amphipods=amphipods):
    letter, (x,y), _ = amphipod
    column = (ord(letter)-65)*2 + 3
    
    holes = set(range(1,5))
    
    for char, (col,h), _ in amphipods:
        if (char == letter) and (col == column) and (y != h):
            holes -= {h}
    
    return (column, min(holes))


def heuristic(amphipod, amphipods=amphipods):
    letter, (x,y), _ = amphipod
    column = (ord(letter)-65)*2 + 3
    
    holes = set(range(1,5))
    
    for char, (col,h), _ in amphipods:
        if (char == letter) and (col == column) and (y != h):
            holes -= {h}
    try:
        return (column, min(holes))
    except:
        print(amphipod)
        print(holes)
        print(amphipods)
        print()
        raise Exception()


def connected(pt, occupied, spaces=spaces):
    pts = adjacent(pt) - occupied
    new = pts.copy()
    
    while new:
        temp = set()
        for p in new:
            temp |= adjacent(p) - occupied
        
        new = temp - pts
        pts |= temp
    
    return pts


def possible(amphipod, amphipods=amphipods, spaces=spaces, doorways=doorways):
    char, pt, moved = amphipod
    occupied = occupado(amphipods)
    home = destination(amphipod, amphipods)
    conn = connected(pt, occupied, spaces)
    poss = set()
    
    for x,y in conn:
        if y < 5:
            if (x,y) == home:
                poss.add((x,y))
            else:
                continue
        elif ((x,y) not in doorways):
            poss.add((x,y))
    
    if moved:
        return {home} if home in poss else set()
    else:
        return poss


def next_states(amphipod_state, spaces=spaces, doorways=doorways):
    new = set()
    t, n, c, *A = amphipod_state
    
    for a in A:
        letter, pt, moveable = a
        for p in possible(a, A, spaces, doorways):
            new_cost = c + cost(letter, pt, p)
            new.add(
                hash_state(
                    (set(A) - {a}) | {(letter, p, True)}, new_cost, n+1
                )
            )
    return new


def finito(amphipods):
    for a in amphipods:
        _, pt, _ = a
        if destination(a, amphipods) != pt:
            return False
    else:
        return True
    


def hash_state(amphipods, c, n):
    
    t = c
    
    for a in amphipods:
        char, pt, moved = a
        t += cost(char, pt, heuristic(a, amphipods))
    
    return tuple([t, n, c] + sorted(amphipods))

In [29]:
amphipods = {('D', (7, 1), False), ('D', (9, 1), True), ('A', (9,2), False), ('B', (7,2), True)}
amphipods = {('A', (5, 1), False)}
first_state = hash_state(amphipods, 0, 0)

In [30]:
first_state = hash_state(amphipods, 0, 0)

In [31]:
next_states(first_state)

{(10, 1, 5, ('A', (4, 5), True)),
 (10, 1, 10, ('A', (3, 1), True)),
 (12, 1, 5, ('A', (6, 5), True)),
 (12, 1, 7, ('A', (2, 5), True)),
 (14, 1, 8, ('A', (1, 5), True)),
 (16, 1, 7, ('A', (8, 5), True)),
 (20, 1, 9, ('A', (10, 5), True)),
 (22, 1, 10, ('A', (11, 5), True))}

In [32]:
destination(list(amphipods)[0], amphipods)

(3, 1)

In [33]:
finito(amphipods)

False

In [43]:
import time

In [44]:
t = time.time()

states = PriorityQueue()
seen = set()
finished = False
best = 1e10

first_state = hash_state(amphipods, 0, 0)
states.put(first_state)

i = 0

while not finished:
    i += 1
    state = states.get_nowait()
        
    if state[2] >= best:
        finished = True
    
    new = next_states(state)
    for n in new:
        if finito(n[3:]):
            if n[2] < best:
                fin = n
                best = n[2]
            break
        elif n[3:] not in seen:
            seen.add(n[3:])
            states.put(n)
        else:
            pass

print(time.time() - t)

17.63256812095642


In [16]:
i

90768

In [15]:
fin

(49532,
 30,
 49532,
 ('A', (3, 1), True),
 ('A', (3, 2), True),
 ('A', (3, 3), True),
 ('A', (3, 4), True),
 ('B', (5, 1), True),
 ('B', (5, 2), True),
 ('B', (5, 3), True),
 ('B', (5, 4), True),
 ('C', (7, 1), True),
 ('C', (7, 2), True),
 ('C', (7, 3), True),
 ('C', (7, 4), True),
 ('D', (9, 1), True),
 ('D', (9, 2), True),
 ('D', (9, 3), True),
 ('D', (9, 4), True))

In [406]:
finito(fin[3:])

True

In [407]:
fin

(43791,
 24,
 43791,
 ('A', (3, 1), False),
 ('A', (3, 2), True),
 ('A', (3, 3), True),
 ('A', (3, 4), True),
 ('B', (5, 1), True),
 ('B', (5, 2), True),
 ('B', (5, 3), True),
 ('B', (5, 4), True),
 ('C', (7, 1), False),
 ('C', (7, 2), True),
 ('C', (7, 3), True),
 ('C', (7, 4), True),
 ('D', (9, 1), True),
 ('D', (9, 2), True),
 ('D', (9, 3), True),
 ('D', (9, 4), True))

In [330]:
list(next_states(list(new)[0]))[0][3:] in seen

True

In [315]:
amphipods

{('A', (7, 1), False),
 ('A', (7, 4), False),
 ('A', (9, 2), False),
 ('A', (9, 3), False),
 ('B', (3, 3), False),
 ('B', (5, 1), False),
 ('B', (5, 4), False),
 ('B', (7, 2), False),
 ('C', (3, 4), False),
 ('C', (5, 2), False),
 ('C', (5, 3), False),
 ('C', (9, 1), False),
 ('D', (3, 1), False),
 ('D', (3, 2), False),
 ('D', (7, 3), False),
 ('D', (9, 4), False)}

---